In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
import contractions
from nltk.corpus import stopwords
from string import punctuation

In [2]:
df = pd.read_csv('movie.csv')
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [3]:
df.shape

(40000, 2)

In [4]:
df.isna().sum()

text     0
label    0
dtype: int64

In [5]:
df.duplicated().sum()

277

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df['label'].value_counts()

label
1    19908
0    19815
Name: count, dtype: int64

In [8]:
def remove_html_tags(x):
    text = re.sub('<.+>','',x)
    return text

def remove_links(x):
    pattern = 'https?://\S*|www\.\S*'
    text = re.sub(pattern,'',x)
    return text

<>:6: SyntaxWarning: invalid escape sequence '\S'
<>:6: SyntaxWarning: invalid escape sequence '\S'
C:\Users\kapil\AppData\Local\Temp\ipykernel_6740\2681458835.py:6: SyntaxWarning: invalid escape sequence '\S'
  pattern = 'https?://\S*|www\.\S*'


In [9]:
df['text'] = df['text'].apply(remove_html_tags)
df['text'] = df['text'].apply(remove_links)
df['text'] = df['text'].str.lower()

df['text'] = df['text'].apply(lambda x:contractions.fix(x))

In [10]:
stop_word_list = stopwords.words('english')

In [11]:
stop_words = []
for i in stop_word_list:
    if i not in ["nor",'no','not',]:
        stop_words.append(i)

stop_words

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 

In [12]:
def remove_punctuations(x):
    string = ''
    for i in x:
        if i not in punctuation:
            string = string + i
    return string

In [13]:
df['clean_text'] = df['text'].apply(remove_punctuations)

In [14]:
def remove_stopwords(x):
    l1 = []
    for i in x.split(' '):
        if i not in stop_words:
            l1.append(i)
    return " ".join(l1)

In [15]:
df['clean_text'] = df['clean_text'].apply(remove_stopwords)

In [17]:
df['clean_text'] = df['clean_text'].apply(lambda x:word_tokenize(x))

In [20]:
df['total_words'] = df['clean_text'].apply(lambda x:len(x))
df['total_char'] = df['clean_text'].apply(lambda x:len(" ".join(x)))

In [24]:
df['total_sent'] = df['text'].apply(lambda x:len(sent_tokenize(x)))

In [26]:
from nltk.stem import PorterStemmer

In [31]:
def stemming(x):
    l = []
    stemmer = PorterStemmer()
    for i in x:
        l.append(stemmer.stem(i))
    return l

In [34]:
df['clean_text'] = df['clean_text'].apply(stemming)

In [36]:
df.to_csv('clean_reviews.csv',index = False)

,text,label,clean_text,total_words,total_char,total_sent
0,i grew up (b. 1965) watching and loving the th...,0,"[grew, b, 1965, watch, love, thunderbird, mate...",89,622,17
1,"when i put this movie in my dvd player, and sa...",0,"[put, movi, dvd, player, sat, coke, chip, expe...",94,649,8
2,why do people who do not know what a particula...,0,"[peopl, not, know, particular, time, past, lik...",96,623,9
3,even though i have great interest in biblical ...,0,"[even, though, great, interest, biblic, movi, ...",34,216,5
4,i am a die hard dads army fan and nothing will...,1,"[die, hard, dad, armi, fan, noth, ever, chang,...",57,335,5
...,...,...,...,...,...,...
39995,"""western union"" is something of a forgotten cl...",1,"[western, union, someth, forgotten, classic, w...",70,535,6
39996,this movie is an incredible piece of work. it ...,1,"[movi, incred, piec, work, explor, everi, nook...",27,192,5
39997,my wife and i watched this movie because we pl...,0,"[wife, watch, movi, plan, visit, sicili, strom...",65,531,6
39998,"when i first watched flatliners, i was amazed....",1,"[first, watch, flatlin, amaz, necessari, featu...",17,130,2
